<a href="https://colab.research.google.com/github/Okelo123/Predict-Patient-Readmission-plpweek5AI/blob/main/week5plp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q streamlit pyngrok scikit-learn pandas numpy pyngrok

In [18]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# ---------------------
# TRAINING THE MODEL
# ---------------------
def train_model():
    np.random.seed(42)
    n_samples = 500
    data = pd.DataFrame({
        'age': np.random.randint(30, 90, n_samples),
        'gender': np.random.choice(['Male', 'Female'], n_samples),
        'num_prev_admissions': np.random.randint(0, 10, n_samples),
        'length_of_stay': np.random.randint(1, 20, n_samples),
        'has_chronic_illness': np.random.choice([0, 1], n_samples, p=[0.4, 0.6]),
        'discharge_type': np.random.choice(['Home', 'Rehab', 'Nursing'], n_samples),
        'readmitted_30_days': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    })

    data_encoded = pd.get_dummies(data, columns=['gender', 'discharge_type'], drop_first=True)
    X = data_encoded.drop('readmitted_30_days', axis=1)
    y = data_encoded['readmitted_30_days']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LogisticRegression(class_weight='balanced')
    model.fit(X_scaled, y)

    return model, scaler, X.columns

# Train model
model, scaler, feature_names = train_model()

# ---------------------
# Streamlit UI
# ---------------------
st.set_page_config(page_title="Patient Readmission Predictor", layout="centered")
st.title("🏥 Predict Patient Readmission (within 30 Days)")
st.markdown("Fill in the patient details below to check the readmission risk.")

# Form
age = st.slider("Age", 30, 90, 45)
gender = st.selectbox("Gender", ["Male", "Female"])
num_prev_admissions = st.slider("Number of Previous Admissions", 0, 10, 1)
length_of_stay = st.slider("Length of Stay (days)", 1, 30, 5)
has_chronic = st.radio("Chronic Illness", ["Yes", "No"])
discharge_type = st.selectbox("Discharge Type", ["Home", "Rehab", "Nursing"])

# Predict Button
if st.button("Predict Readmission Risk"):
    input_data = pd.DataFrame({
        'age': [age],
        'num_prev_admissions': [num_prev_admissions],
        'length_of_stay': [length_of_stay],
        'has_chronic_illness': [1 if has_chronic == "Yes" else 0],
        'gender_Male': [1 if gender == "Male" else 0],
        'discharge_type_Nursing': [1 if discharge_type == "Nursing" else 0],
        'discharge_type_Rehab': [1 if discharge_type == "Rehab" else 0],
    })

    for col in feature_names:
        if col not in input_data.columns:
            input_data[col] = 0
    input_data = input_data[feature_names]

    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1]

    if prediction == 1:
        st.error(f"⚠️ High Risk: Patient likely to be readmitted (Confidence: {probability:.2f})")
    else:
        st.success(f"✅ Low Risk: Patient not likely to be readmitted (Confidence: {1 - probability:.2f})")


Overwriting app.py


In [19]:
from pyngrok import ngrok
from google.colab import userdata

# Kill any running tunnels
ngrok.kill()

# Get your ngrok authtoken from Colab Secrets
# Replace 'NGROK_AUTHTOKEN' with the name you used for your secret in the Secrets panel
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')

# Set the authtoken
ngrok.set_auth_token(ngrok_authtoken)

# Launch
# Revert ngrok.connect and add bind_tls=True
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("Streamlit app running at:", public_url)

!streamlit run app.py &>/content/logs.txt &

Streamlit app running at: NgrokTunnel: "https://66ab-35-203-153-248.ngrok-free.app" -> "http://localhost:8501"


In [11]:
!cat /content/logs.txt

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
